In [ ]:
!pip install fastmcp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.3/201.3 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.6/84.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.9/155.9 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.4/96.4 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 12.9 MB/s eta 0:00:00


In [ ]:
!pip install qwen_vl_utils

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.7/39.7 MB 16.6 MB/s eta 0:00:00


In [ ]:
pip install pyngrok

In [ ]:

# 🛠 2. Set up ngrok (replace with your own token)
NGROK_TOKEN = "2zfc19GGusf8X8tanJrTodzeneJ_6oCzBYUnsTkjH7wekVz8E"

from pyngrok import ngrok, conf
conf.get_default().auth_token = NGROK_TOKEN

# 📦 3. Minimal MedVLM singleton + FastMCP server
import io, uuid, base64, threading
from typing import Annotated
from fastmcp import FastMCP, Context
from PIL import Image
import torch
from transformers import (
    AutoProcessor, Qwen2VLForConditionalGeneration, GenerationConfig,
)
from qwen_vl_utils import process_vision_info
MODEL_ID  = "JZPeterPan/MedVLM-R1"
_DEVICE   = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class _MedVLM:
    _model, _proc = None, None
    _gen_cfg = GenerationConfig(max_new_tokens=256, do_sample=False,
                                temperature=1.0, pad_token_id=151_643)
    @classmethod
    def _load(cls):
        if cls._model: return
        cls._model = Qwen2VLForConditionalGeneration.from_pretrained(
            MODEL_ID, torch_dtype=torch.bfloat16, device_map="auto",
            # attn_implementation="flash_attention_2",
        ).eval()
        cls._proc  = AutoProcessor.from_pretrained(MODEL_ID)

    @classmethod
    def infer(cls, img_bytes: bytes, question: str) -> str:
        from qwen_vl_utils import process_vision_info          # same helper as earlier
        cls._load()
        img = Image.open(io.BytesIO(img_bytes)).convert("RGB")
        payload = [{"role":"user","content":[
            {"type":"image","image":img},
            {"type":"text","text":f"{question or 'Describe the medically relevant findings.'}\n\n"
                                  "1. Think step-by-step inside <think>…</think>.\n"
                                  "2. Put the answer inside <answer>…</answer>."}
        ]}]
        text = cls._proc.apply_chat_template(payload, tokenize=False,
                                             add_generation_prompt=True)
        imgs, vids = process_vision_info(payload)
        inputs = cls._proc(text=text, images=imgs, videos=vids,
                           return_tensors="pt", padding=True).to(_DEVICE)
        out = cls._model.generate(**inputs, generation_config=cls._gen_cfg,
                                  use_cache=True)[0][len(inputs.input_ids[0]):]
        return cls._proc.decode(out, skip_special_tokens=True,
                                clean_up_tokenization_spaces=False)

# Load model
print("Start loading model")
_MedVLM = _MedVLM()
_MedVLM._load()
print("Finished loading model")
# ---------- Fast-MCP wiring ----------
mcp = FastMCP(name="MedVLM-Server")
_blob: dict[str, bytes] = {}

@mcp.resource("upload://{id}")
def read_blob(id: str, ctx: Context) -> bytes:
    # if id not in _blob: raise FileNotFound(f"unknown {id}")
    return _blob[id]

@mcp.tool
def upload_image(b64: Annotated[str, "base-64 file bytes"]) -> str:
    bid = str(uuid.uuid4()); _blob[bid] = base64.b64decode(b64)
    return f"upload://{bid}"



@mcp.tool
def medvlm_infer(image_uri: Annotated[str, "`upload://…`"],
                 question: Annotated[str|None, "optional prompt"]=None) -> str:
    bid = image_uri.removeprefix("upload://")
    # if bid not in _blob: raise FileNotFound(f"unknown {image_uri}")
    return _MedVLM.infer(_blob[bid], question or "")

# 🔌 4. Run server in background
def _serve(): mcp.run(transport="http", host="0.0.0.0",port=8000)
threading.Thread(target=_serve, daemon=True).start()
# 🌐 5. Expose via ngrok
public_url = ngrok.connect(8000, "http").public_url
print("🚀 Public MCP endpoint:", public_url)

Start loading model


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/4.42G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

...

Finished loading model


╭─ FastMCP 2.0 ──────────────────────────────────────────────────────────────╮
│                                                                            │
│        _ __ ___ ______           __  __  _____________    ____    ____     │
│       _ __ ___ / ____/___ ______/ /_/  |/  / ____/ __ \  |___ \  / __ \    │
│      _ __ ___ / /_  / __ `/ ___/ __/ /|_/ / /   / /_/ /  ___/ / / / / /    │
│     _ __ ___ / __/ / /_/ (__  ) /_/ /  / / /___/ ____/  /  __/_/ /_/ /     │
│    _ __ ___ /_/    \__,_/____/\__/_/  /_/\____/_/      /_____(_)____/      │
│                                                                            │
│                                                                            │
│                                                                            │
│    🖥️  Server name:     MedVLM-Server                                       │
│    📦 Transport:       Streamable-HTTP                                     │
│    🔗 Server URL:      http://0.0.0.0:8000/mcp/                            │
│                                                                            │
│    📚 Docs:            https://gofastmcp.com                               │
│    🚀 Deploy:          https://fastmcp.cloud                               │
│                                                                            │
│    🏎️  FastMCP version: 2.10.5                                              │
│    🤝 MCP version:     1.11.0                                              │
│                                                                            │
╰────────────────────────────────────────────────────────────────────────────╯

[07/14/25 07:09:45] INFO     Starting MCP server 'MedVLM-Server' with transport 'http' on            ]8;id=373422;file:///usr/local/lib/python3.11/dist-packages/fastmcp/server/server.py\server.py]8;;\:]8;id=332837;file:///usr/local/lib/python3.11/dist-packages/fastmcp/server/server.py#1448\1448]8;;\
                             http://0.0.0.0:8000/mcp/                                                              

/usr/local/lib/python3.11/dist-packages/websockets/legacy/__init__.py:6: DeprecationWarning: websockets.legacy is deprecated; see https://websockets.readthedocs.io/en/stable/howto/upgrade.html for upgrade instructions
  warnings.warn(  # deprecated in 14.0 - 2024-11-09
/usr/local/lib/python3.11/dist-packages/uvicorn/protocols/websockets/websockets_impl.py:17: DeprecationWarning: websockets.server.WebSocketServerProtocol is deprecated
  from websockets.server import WebSocketServerProtocol
INFO:     Started server process [195]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


🚀 Public MCP endpoint: https://471284188f4a.ngrok-free.app


In [ ]:
# !pip uninstall -y flash-attn
# !pip install flash-attn --no-build-isolation --no-clean